In [78]:
import numpy as np
import open3d as o3d
from e_slope import preprocess_point_cloud

In [79]:
pcd1 = o3d.io.read_point_cloud('Plane.1.pcd')
pcd2 = o3d.io.read_point_cloud('Plane.2.pcd')
pcd3 = o3d.io.read_point_cloud('Plane.3.pcd')
box1 = o3d.io.read_point_cloud('Box.pcd')
box2 = o3d.io.read_point_cloud('Box.sampled.pcd')
box = box1 + box2
pcd = pcd1 + pcd2 + pcd3
# pcd = preprocess_point_cloud(pcd,voxel_size=0.03)
pcd.paint_uniform_color([1, 0, 0])

PointCloud with 13073 points.

In [80]:
o3d.visualization.draw_geometries([pcd])

In [81]:
points = np.asarray(pcd.points)
x_range = points[:, 0].max() - points[:, 0].min()
y_range = points[:, 1].max() - points[:, 1].min()
z_range = points[:, 2].max() - points[:, 2].min()
o3d.visualization.draw_geometries([pcd], window_name="Original Point")  # plane
print(f'点的x轴范围：{points[:, 0].min()}~{points[:, 0].max()}')
print(f'点的y轴范围：{points[:, 1].min()}~{points[:, 1].max()}')
print(f'点的z轴范围：{points[:, 2].min()}~{points[:, 2].max()}')

点的x轴范围：-0.5025805234909058~0.512271523475647
点的y轴范围：-0.49930712580680847~1.8308430910110474
点的z轴范围：-0.034461379051208496~0.4484121799468994


In [82]:
# 随机生成每个点在每个轴上的偏移量，范围为1%到10%
x_offsets = np.random.uniform(-0.03 * x_range, 0.03 * x_range, size=points.shape[0])
y_offsets = np.random.uniform(-0.03 * y_range, 0.03 * y_range, size=points.shape[0])
z_offsets = np.random.uniform(-0.03 * z_range, 0.03 * z_range, size=points.shape[0])
# 将偏移量加到原始点云坐标上
points[:, 0] += x_offsets
points[:, 1] += y_offsets
points[:, 2] += z_offsets
pcd.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_geometries([pcd], window_name="Updated Point Cloud")

In [83]:
print(f'随机点后的x轴范围：{points[:, 0].min()}~{points[:, 0].max()}')
print(f'随机点后的y轴范围：{points[:, 1].min()}~{points[:, 1].max()}')
print(f'随机点后的z轴范围：{points[:, 2].min()}~{points[:, 2].max()}')

随机点后的x轴范围：-0.5302672154657813~0.5363210068676371
随机点后的y轴范围：-0.5666854400268722~1.8961777677636122
随机点后的z轴范围：-0.045594914919060106~0.4601276622692495


In [84]:
def bilateral_filter(pcd, sigma_s=0.05, sigma_r=0.05, radius=0.05):
    point_cloud = np.asarray(pcd.points)
    def gaussian(x, sigma):
        return np.exp(-0.5 * (x / sigma) ** 2)

    filtered_points = np.zeros_like(points)
    pcd_tree = o3d.geometry.KDTreeFlann(o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points)))

    for i in range(points.shape[0]):
        [_, idx, _] = pcd_tree.search_radius_vector_3d(points[i], radius)
        neighbors = points[idx, :]

        spatial_weights = gaussian(np.linalg.norm(neighbors - points[i], axis=1), sigma_s)
        intensity_weights = gaussian(np.linalg.norm(neighbors - points[i], axis=1), sigma_r)

        bilateral_weights = spatial_weights * intensity_weights
        bilateral_weights /= np.sum(bilateral_weights)

        filtered_points[i] = np.sum(neighbors * bilateral_weights[:, np.newaxis], axis=0)

    return filtered_points

In [85]:
points = bilateral_filter(pcd)
pcd.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_geometries([pcd], window_name="Filtered Point Cloud")
print(f'滤波后的x轴范围：{points[:, 0].min()}~{points[:, 0].max()}')
print(f'滤波点后的y轴范围：{points[:, 1].min()}~{points[:, 1].max()}')
print(f'滤波点后的z轴范围：{points[:, 2].min()}~{points[:, 2].max()}')

滤波后的x轴范围：-0.5110434338417248~0.5209893491908648
滤波点后的y轴范围：-0.5598775434023994~1.876666665649783
滤波点后的z轴范围：-0.022494306660450946~0.4384442974349859


In [86]:
def segment_planes(pcd, distance_threshold=0.05, ransac_n=5,
                   num_iterations=3000):
    plane_model, inliers = pcd.segment_plane(distance_threshold, ransac_n, num_iterations)
    [a, b, c, d] = plane_model
    normals = np.asarray([a, b, c])
    inlier_cloud = pcd.select_by_index(inliers)
    outlier_cloud = pcd.select_by_index(inliers, invert=True)
    return inlier_cloud, outlier_cloud, normals

In [87]:
inlier_cloud_1, outlier_cloud_1, normal_vector_1 = segment_planes(pcd)  # plane
inlier_cloud_2, outlier_cloud_2, normal_vector_2 = segment_planes(outlier_cloud_1)
inlier_cloud_3, remaining_cloud, normal_vector_3 = segment_planes(outlier_cloud_2)

In [88]:
def calculate_z_mean(point_cloud):
    """Calculate the mean of the Z-axis values of a point cloud."""
    points = np.asarray(point_cloud.points)
    return np.mean(points[:, 2])

In [89]:
def classify_point_clouds(pcds, normals):
    """Classify point clouds based on the mean Z value."""
    clusters = []
    for pc, normal in zip(pcds, normals):
        z_mean = calculate_z_mean(pc)
        clusters.append((z_mean, pc, normal))

    # Sort clusters based on the Z mean value
    clusters_sorted = sorted(clusters, key=lambda x: x[0], reverse=True)

    return clusters_sorted

In [90]:
pcds = [inlier_cloud_1, inlier_cloud_2, inlier_cloud_3]
normal_vectors = [normal_vector_1, normal_vector_2, normal_vector_3]
#
classified_clusters = classify_point_clouds(pcds, normal_vectors)

# 动态创建变量名的字典
clusters = {}
normal_vectors_dict = {}
# Output classification results and assign clusters and normal vectors
for i, (z_mean, cluster, normal_vector) in enumerate(classified_clusters, start=1):
    # Visualize point cloud (optional)
    # o3d.visualization.draw_geometries([cluster])
    clusters[f'cluster_{i}'] = cluster
    normal_vectors_dict[f'normal_vector_{i}'] = normal_vector

inlier_cloud_1 = clusters['cluster_1']
inlier_cloud_2 = clusters['cluster_2']
inlier_cloud_3 = clusters['cluster_3']

normal_vector_1 = normal_vectors_dict['normal_vector_1']
normal_vector_2 = normal_vectors_dict['normal_vector_2']
normal_vector_3 = normal_vectors_dict['normal_vector_3']

In [91]:
# # 显示分割结果
inlier_cloud_1.paint_uniform_color([1.0, 0, 0])  # 将第一个平面内的点涂成红色
inlier_cloud_2.paint_uniform_color([0, 1.0, 0])  # 将第二个平面内的点涂成绿色
inlier_cloud_3.paint_uniform_color([1, 1, 0])  # 将第三个平面内的点涂成黄色
remaining_cloud.paint_uniform_color([0, 0, 1.0])  # 将剩余点涂成蓝色
o3d.visualization.draw_geometries([inlier_cloud_1, inlier_cloud_2, inlier_cloud_3,remaining_cloud],
                                  window_name="Point Segmentation")  # plane

In [92]:
def get_slope(normal_1, normal_2):
    cos_theta = np.dot(normal_1, normal_2) / (
            np.linalg.norm(normal_1) * np.linalg.norm(normal_2))
    angle = np.arccos(cos_theta)
    angle_degrees = np.degrees(angle)
    return angle_degrees

In [93]:
# # 输出两个平面的法向量
print(f"First plane normal vector: {normal_vector_1}")
print(f"Second plane normal vector: {normal_vector_2}")
print(f"Third plane normal vector: {normal_vector_3}")
#
angle_pred12 = get_slope(normal_vector_1, normal_vector_2)
angle_pred23 = get_slope(normal_vector_2, normal_vector_3)
print(f"Angle_pred between the 1-2 planes: {angle_pred12} degrees")
print(f"Angle_pred between the 2-3 planes: {angle_pred23} degrees")

normal_1 = np.array([-0.0152804, 0.182808, 0.98303])  # plane
normal_2 = np.array([-0.020418, -0.583249, 0.812037])
normal_3 = np.array([0, 0, 1])  # plane

angle_truth12 = get_slope(normal_1, normal_2)
angle_truth23 = get_slope(normal_2, normal_3)
print(f"Angle_truth between the 1-2 planes: {angle_truth12} degrees")
print(f"Angle_truth between the 2-3 planes: {angle_truth23} degrees")
angle_error12 = abs(angle_truth12 - angle_pred12) / angle_truth12 * 100
angle_error23 = abs(angle_truth23 - angle_pred23) / angle_truth23 * 100
print(f"Error between 1-2 planes: {angle_error12} %")
print(f"Error between 2-3 planes: {angle_error23} %")

First plane normal vector: [-0.01489073  0.15149347  0.98834609]
Second plane normal vector: [-0.01799653 -0.56039931  0.82802701]
Third plane normal vector: [-0.00428828 -0.04943856  0.99876796]
Angle_pred between the 1-2 planes: 42.79843622391565 degrees
Angle_pred between the 2-3 planes: 31.263527688030308 degrees
Angle_truth between the 1-2 planes: 46.21565577111552 degrees
Angle_truth between the 2-3 planes: 35.704583203040684 degrees
Error between 1-2 planes: 7.394073480475441 %
Error between 2-3 planes: 12.438334568297567 %
